<a target="_blank" href="https://colab.research.google.com/github/ArtificialIntelligenceToolkit/aitk/blob/master/notebooks/Robotics/SeekLight.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install aitk --upgrade --quiet

In [1]:
import aitk.robots
aitk.robots.__version__

'0.8.2'

In [2]:
from random import randrange

In [3]:
# Try using the worlds: LightBehindWall, LightInCorral, LightEnclosed, or LightInMaze (this one is challenging)

In [4]:
world = aitk.robots.load_world("LightEnclosed")

Loading ./worlds/LightEnclosed.json...
Using random seed: 2121138


In [5]:
robot = world.robots[0]

In [6]:
world.watch()

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

In [7]:
for robot in world.robots:
    for device in robot:
        device.watch()

In [8]:
def seekLight(robot):
    left_ir = robot["left-ir"].get_distance()
    right_ir = robot["right-ir"].get_distance()
    left_light = robot["left-light"].get_reading()
    right_light = robot["right-light"].get_reading()
    total_light = left_light + right_light
    diff_light = left_light - right_light
    if total_light > 2.5:
        print("\nFound light!")
        return True
    elif left_ir == robot[0].get_max() and right_ir == robot[1].get_max() and \
        robot.state["timer"] == 0:
        if abs(diff_light) <= robot.state["light-diff"]:
            # move forward
            robot.move(0.5, 0)
            #if robot.state["debug"]: print("F",end="")
        elif diff_light < 0:
            # light stronger to right
            robot.move(0.2, -0.1)
            #if robot.state["debug"]: print("r",end="")
        else:
            # light stronger to left
            robot.move(0.2, 0.1)
            #if robot.state["debug"]: print("l", end="")
    elif robot.state["timer"] > 0 and robot.state["timer"] < 5:
        # timer triggered, continue current rotation
        robot.state["timer"] += 1
        #if robot.state["debug"]: print("T",end="")
    elif left_ir < robot[0].get_max():
        # obstacle on left, turn right, trigger timer
        robot.move(0.1, -0.4)
        robot.state["timer"] = 1
        #if robot.state["debug"]: print("R",end="")
    elif right_ir < robot[1].get_max():
        # obstacle on right, turn left, trigger timer
        robot.move(0.1, 0.4)
        robot.state["timer"] = 1
        #if robot.state["debug"]: print("L",end="")
    else:
        # reset timer to zero
        robot.state["timer"] = 0
        #if robot.state["debug"]: print("t",end="")
            

In [9]:
world.robots[0].set_random_pose()
world.update()

In [10]:
robot.state["debug"] = True
robot.state["timer"] = 0
robot.state["light-diff"] = 0.02

robot.set_max_trace_length(120)

#world.run([seekLight])
world.seconds(120, [seekLight], real_time=True)

  0%|          | 0/1200 [00:00<?, ?it/s]


Found light!
Simulation stopped at: 00:00:52.4; speed 0.98 x real time
